# Zadání zápočtové úlohy z předmětu 01ZLMA 2022/2023

https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients

Data Set Information:

This research aimed at the case of customers default payments in Taiwan and compares the predictive accuracy of probability of default among six data mining methods. From the perspective of risk management, the result of predictive accuracy of the estimated probability of default will be more valuable than the binary result of classification - credible or not credible clients





In [1]:
%pip install pandas matplotlib xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls"
df = pd.read_excel(url, header=1)

In [3]:
df = df.rename(columns = {"default payment next month": "default"})

In [4]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [5]:
df = df.rename(columns=lambda x: x.lower())

In [6]:
df.shape

(30000, 25)

In [7]:
bill_cols = list(filter(lambda col: col.find("bill_amt") != -1, df.columns))

In [8]:
pay_cols = list(filter(lambda col: col.find("pay_amt") != -1, df.columns))

In [9]:
df['bill_min'] = df[bill_cols].apply(min, axis=1)

In [10]:
df['bill_max'] = df[bill_cols].apply(max, axis=1)

In [11]:
df['pay_min'] = df[pay_cols].apply(min, axis=1)

In [12]:
df['pay_max'] = df[pay_cols].apply(max, axis=1)

In [13]:
df = df.drop(pay_cols + bill_cols + ['id'] + ['pay_' + str(i) for i in range(2,6)], axis=1)

In [14]:
df.head()

,limit_bal,sex,education,marriage,age,pay_0,pay_6,default,bill_min,bill_max,pay_min,pay_max
0,20000,2,2,1,24,2,-2,1,0,3913,0,689
1,120000,2,2,2,26,-1,2,1,1725,3455,0,2000
2,90000,2,2,2,34,0,0,0,13559,29239,1000,5000
3,50000,2,2,1,37,0,0,0,28314,49291,1000,2019
4,50000,1,2,1,57,-1,0,0,5670,35835,679,36681


In [15]:
df = df[(df['bill_min'] > 0) &
        (df['pay_max'] > 0) &
        (df['marriage'] != 0) & 
        (df['marriage'] != 3) &
        (df['education'] != 0)]

In [16]:
df.loc[df['education'] > 3, 'education'] = 3

In [17]:
df['education'] = pd.Categorical(df['education'])

In [18]:
df['sex'] = pd.Categorical(df['sex'])

In [19]:
df['marriage'] = pd.Categorical(df['marriage'])

In [20]:
df.describe()

,limit_bal,age,pay_0,pay_6,default,bill_min,bill_max,pay_min,pay_max
count,21943.000000,21943.000000,21943.000000,21943.000000,21943.000000,21943.000000,21943.000000,21943.000000,2.194300e+04
mean,162801.334366,35.271977,0.085996,0.021328,0.222167,43168.164563,73786.276307,1665.886706,1.634583e+04
std,130553.935668,9.221242,1.042020,0.947823,0.415712,59592.725589,82413.780752,2748.702698,3.571387e+04
min,10000.000000,21.000000,-2.000000,-2.000000,0.000000,1.000000,12.000000,0.000000,1.500000e+01
25%,50000.000000,28.000000,0.000000,0.000000,0.000000,4596.500000,19652.500000,0.000000,2.843000e+03
50%,130000.000000,34.000000,0.000000,0.000000,0.000000,19852.000000,47637.000000,700.000000,5.300000e+03
75%,230000.000000,41.000000,0.000000,0.000000,0.000000,55951.500000,97714.500000,2050.000000,1.300000e+04
max,1000000.000000,79.000000,8.000000,8.000000,1.000000,551702.000000,983931.000000,63758.000000,1.227082e+06


In [21]:
df_table = df[['default', 
               'age', 
               'sex', 
               'education', 
               'marriage', 
               'limit_bal']].rename(columns={'age':'age_full', 'limit_bal':'limit_bal_full'})

In [22]:
age_breaks = [-np.infty, 25, 30, 35, 40, np.infty]
age_lables = ['<25', '25-30', '30-35','35-40', '>40']
df_table['age'] = pd.cut(df_table['age_full'], bins=age_breaks, labels=age_lables)

In [23]:
limit_bal_breaks = [-np.infty,50_000, 100_000, 150_000, 200_000, np.infty]
limit_bal_lables = ['<50k','50k-100k','100k-150k','150k-200k','>200k']
df_table['limit_bal'] = pd.cut(df_table['limit_bal_full'], bins=limit_bal_breaks, labels=limit_bal_lables)

In [24]:
df_table = df_table.drop(filter(lambda x: x.find("_full") != -1, df_table.columns), axis=1)

In [25]:
df_table_grouped = df_table.groupby(['age', 'limit_bal', 'sex'], dropna=False)
df_table = df_table_grouped.agg({'default':'sum'}).rename(columns = {'default':'default_yes'})

In [26]:
df_table['default_no'] = df_table_grouped.size() - df_table['default_yes']

In [27]:
df_table.head()

default_yes  default_no
age limit_bal sex                         
<25 <50k      1            210         444
              2            326         710
    50k-100k  1             23          94
              2            146         439
    100k-150k 1             12          43

In [28]:
%pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
from sklearn.model_selection import train_test_split

In [30]:
train_data, test_data = train_test_split(df, test_size = 0.3, random_state=42, stratify=df['default'])

In [31]:
train_data.shape

(15360, 12)

In [32]:
test_data.shape

(6583, 12)

## Zadání

## 01 - Grafické zobrazení dat a jejich kontrola

Použijte dataset `data_train` a `data_test`. 

* Navrhněte vhodný způsob zobrazení dat, který vám pomůže určit, zdali použití logistické regrese pro predikci nesplacení úvěru je možné a vhodné při použítí zvolených trénovacích a testvacích dat.
* Vykreslete boxploty pro různé numerické proměnné, odezvu `default` a barevně odlište kategorické proměnné. Je z těchto obrázků možné odhadnout jaká proměnná bude mít vliv na odezvu? (jen `data_train`).
* Pro vybrané skupiny z předešlé otázky navrhněte vhodný statistický test, proveďte ho a řekněte zdali je vámi zvolená proměnná statisticky významná pro určení schopnosti splácet (jen `data_train`). 

## 02 - Logistická regrese agregovaná tabulková data

použijte `data_table`





* Vytvořte odezvu pro logistický model binomického rozdělení a sestrojte nulový model. Jaká je ve zkoumané populaci  průměrná šance na neschopnost splácet úvěr  a jaká je pravděpodobnost nesplacení úvěru? 

* Sestrojte model, kde závisí výskyt nesplacení úvěru pouze na proměnné `limit_bal`. Je vliv limitu významná proměnná na hladině významnosti 0.05? Pokud ano, zjistěte, kolikrát mají lidé s limitem větším než 100 vetší šanci na neschopnost splácet než lidé s limitem nad 200.

* Sestrojte model, kde závisí výskyt neschopnosti splácet pouze na věku `age`.  Je věk  statisticky významná proměnná na hladině významnosti 0.01? Pokud ano, zjistěte, kolikrát mají pacienti ve věku mezi 35-40 menší šanci na  neschopnost splácet než lidé ve věku 20-25.

* Předpokládejte, že šance na neschopnost splácet roste exponenciálně s hodnotou limitu a klesá exponenciálně s věkem. Vytvořte příslušné numerické spojité prediktory. Sestrojte model, kde šance na neschopnost splácet bude záležet na numerických hodnotách tlaku a věku (bez interakce). Jaký je poměr šancí na neschopnost splácet mezi lidmi lišícími se limitem o 50000 a majícími stejný věk? 

* Otestujte předešlý model proti saturovanému modelu. Má tento test opodstatnění? Okomentujte jeho výsledek. 

In [38]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [37]:
df_table.head()

default_yes  default_no
age limit_bal sex                         
<25 <50k      1            210         444
              2            326         710
    50k-100k  1             23          94
              2            146         439
    100k-150k 1             12          43

In [48]:
df_table.columns

Index(['default_yes', 'default_no'], dtype='object')

In [50]:
df['default'].describe()

count    21943.000000
mean         0.222167
std          0.415712
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: default, dtype: float64

In [ ]:
X = df1[['intercept']]
y = df['default']
model = sm.Logit(y, X)
result = model.fit()

print(result.summary())
print(result.aic)

In [55]:
#X = df[['age', 'sex', 'education', 'marriage', 'limit_bal']]
X = df[['limit_bal', 'sex', 'education', 'marriage', 'age', 'pay_0', 'pay_6',
    'bill_min', 'bill_max', 'pay_min', 'pay_max']]
y = df['default']
model = sm.Logit(y, X)
result = model.fit()

print(result.summary())
print(result.aic)

Optimization terminated successfully.
         Current function value: 0.442313
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                default   No. Observations:                21943
Model:                          Logit   Df Residuals:                    21932
Method:                           MLE   Df Model:                           10
Date:                Tue, 23 May 2023   Pseudo R-squ.:                  0.1649
Time:                        12:14:02   Log-Likelihood:                -9705.7
converged:                       True   LL-Null:                       -11622.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
limit_bal  -1.769e-06   2.08e-07     -8.506      0.000   -2.18e-06   -1.36e-06
sex           -0.1782      0.

Už z tohoto výsledku lze usoudit, že proměnné `age` a `pay_max` nejsou významné pro tento model.

In [79]:
X = df[['limit_bal', 'sex', 'education', 'marriage', 'pay_0', 'pay_6',
    'bill_min', 'pay_min']]
y = df['default']
model = sm.Logit(y, X)
result = model.fit()

print(result.summary())
print(result.aic)

Optimization terminated successfully.
         Current function value: 0.442537
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                default   No. Observations:                21943
Model:                          Logit   Df Residuals:                    21935
Method:                           MLE   Df Model:                            7
Date:                Tue, 23 May 2023   Pseudo R-squ.:                  0.1645
Time:                        12:57:27   Log-Likelihood:                -9710.6
converged:                       True   LL-Null:                       -11622.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
limit_bal  -2.038e-06   1.87e-07    -10.878      0.000   -2.41e-06   -1.67e-06
sex           -0.1897      0.

In [58]:
model_1 = smf.logit('default ~ 1', data=df).fit()

print(model_1.summary())

Optimization terminated successfully.
         Current function value: 0.529636
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                default   No. Observations:                21943
Model:                          Logit   Df Residuals:                    21942
Method:                           MLE   Df Model:                            0
Date:                Tue, 23 May 2023   Pseudo R-squ.:               8.452e-11
Time:                        12:18:27   Log-Likelihood:                -11622.
converged:                       True   LL-Null:                       -11622.
Covariance Type:            nonrobust   LLR p-value:                       nan
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.2531      0.016    -77.163      0.000      -1.285      -1.221


In [59]:
formula_satur = 'default ~ limit_bal * sex * education * marriage * age * pay_0 * pay_6 * bill_min * bill_max * pay_min * pay_max' 
#model_satur = smf.logit(formula_satur, data=df).fit()

#print(model_satur.summary())

/usr/local/lib/python3.10/dist-packages/statsmodels/discrete/discrete_model.py:1819: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/usr/local/lib/python3.10/dist-packages/statsmodels/discrete/discrete_model.py:1819: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/usr/local/lib/python3.10/dist-packages/statsmodels/discrete/discrete_model.py:1819: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/usr/local/lib/python3.10/dist-packages/statsmodels/discrete/discrete_model.py:1819: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))


LinAlgError: ignored

Na saturovany model nemame data :/ (prilis mnoho interakci prilis vysokeho radu)

In [69]:
formula_order_1 = 'default ~ limit_bal + sex + education + marriage + age + \
                pay_0 + pay_6 + bill_min + bill_max + pay_min + pay_max '
my_interactions = ' + limit_bal * bill_max * pay_max'
formula_interactions = formula_order_1 + my_interactions

model_interactions = smf.logit(formula_interactions, data=df).fit()

print(model_interactions.summary())
print(model_interactions.aic)

Optimization terminated successfully.
         Current function value: 0.440400
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                default   No. Observations:                21943
Model:                          Logit   Df Residuals:                    21926
Method:                           MLE   Df Model:                           16
Date:                Tue, 23 May 2023   Pseudo R-squ.:                  0.1685
Time:                        12:31:30   Log-Likelihood:                -9663.7
converged:                       True   LL-Null:                       -11622.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                     -0.7755      0.108     -7.177      0.000      

In [35]:

# Perform logistic regression using statsmodels
X = pd.get_dummies(individual_data['Category'], drop_first=True)  
X = sm.add_constant(X)  # Add constant column for intercept
y = pd.get_dummies(individual_data['Group'], drop_first=True) 
model = sm.Logit(y, X)
result = model.fit()

# Print the logistic regression summary
print(result.summary())

AttributeError: ignored

## 03 - Poissonovská regrese na agregovaná tabulková data (4/7)

použijte  `data_table`


* Upravte tabulku do potřebného formátu a sestrojte čistě aditivní loglineární model pro četnosti pozorování ve skupinách, který předpokládá vzájemnou
nezávislost mezi všemi skupinovými prediktory.

* Na základě předchozího modelu, jaký je odhad šance na neschopnost splácet mezi všemi vybranými lidmi ve skupině a jaký je odhad pravděpodobnosti neschopnosti splácet?

* Sestavte model, který obsahuje všechny možné dvojné interakce mezi klasifikačními veličinami a porovnejte tento model s předchozím modelem bez interakcí. Je tento model statisticky významně lepší?

* Sestavte saturovaný model. Je tento model statisticky významně lepší než předchozí s dvojtými interakcemi? Vyberte model který je podle vás nejlepší.

* Na základě vybraného modelu, je ve všech věkových skupinách stejná souvislost mezi limitem a šancí splácet?

* Na základě vybraného modelu, je pro obě pohlaví stejná souvislost mezi limitem a šancí splácet?

* Ve skupině co se týče vzdělání je nějvětší souvislost mezi schopností splácet a limitem?


## 04 - Logistická regrese - statistika (5/8)

Nyní již používejte `data_train`

* Vypište kontingenční tabulku pro proměnné pohlaví (`sex`) a schopnost splácet  (`default`). Ručně z tabulky spočtěte empirický poměr šancí na schopnost splácet (muž vs. žena) a pravděpodobnost onemocnění pro ženu a muže. Výsledek porovnejte s výsledky z logistické regrese, kde jediným prediktorem bude právě proměnná `sex` a odezvou proměnná `default`. Pro poměr šancí uveďte i 95% konfidenční interval a okometujte, zdali mají ženy signifikantně menší šanci na schopnost splácet.

* Vypište kontingenční tabulku pro proměnné vzdělání (`education`) a schopnost splácet (`default`). Ručně z tabulky spočtěte empirický poměr šancí na onemocnění srdce (type 0: asymptomatic, proti ostatním) a pravděpodobnosti onemocnění pro jednotlivé typy. Výsledek porovnejte s výsledky z logistické regrese, kde jediným prediktorem bude právě proměnná `education` a odezvou proměnná `default`. Pro poměr šancí uveďte i 95% konfidenční interval a okometujte, zdali mají s lepším vzděláním mají menší šanci na neschopnost splácet úvěr.

* Sestavte model, kde použijete všechny dostupné proměnné (jak faktorové tak numerické). Pomocí deviančních testů model postupně redukujte. Výsledný model porovnejte s modelem, který byste obdrželi při použití automatického výběru s funkcí `step()`.

* Pro vybraný model spočtěte šanci na schopnost splácet pro muže oproti ženám, včetně 95% konfidenčních intervalů. Stejně tak pro manželství a vzdělání. Jak se výsledek změnil oproti jednoduchým modelům a jak byste tuto změnu vysvětlili?

* Použijte váš model a spočtěte odhad pravděpodobnosti schopnosti splácet pro testovací data vykreslete pro proměnou `limit_bal` predikční konfidenční intervaly. 
Nápověda krom cvičení, také třeba zde: https://fromthebottomoftheheap.net/2017/05/01/glm-prediction-intervals-i/

* Na základě trénovacích dat zvolte vhodný treshold pro určení splatí / nesplatí a na testovacích datech spočtěte Accuracy a vykreslete ROC křivku spolu s AUC.





## 05 - Logistická regrese -  strojové učení

* Sestavte workflow (pipeline) na trenovacích datech pro logistickou regresi s použitím regularizace (elestic-net), která bude obsahovat 
 *   Přípravu proměnných, transformaci, one-hot encoding, normalizaci, ... (Možno použít recipe z tidy models)
 *   Hledání "optimálního" nastavení hyperparametru v regularizaci. 
 *   k-fold cross validaci

* Pomocí předchozí pipline/workflow vyberte hodnotu hyperparametru. Pokud chceme uvedený algoritmus použít k detekci pacientů s nemocí srdce, jakou statistiku bychom měli sledovat abychom omylem nepropustili nemocného domů jako zdravého? 

* Spočtěte a porovnejte statistiky používané při binární klasifikaci na trénovacích a testovacích datech. Vykreslete ROC křivku a spočtěte plochu pod ní jak pro testovací tak trénovací data. Co můžeme říci o kvalitě modelu z této sekce 05 a modelu z předchozí 04?




## Poznámky k odevzdání a vypracování

* Úkol vypracujte ve skupině. Pokud se budete s někým radit mimo skupinu, nezapomeňte u dané odpovědi uvést zdroj vašich informací. 

* Úkol můžete odevzdávat jako spustitelný jupyter notebook na google colab (stačí odevzdat link na github). Před odevzdáním ZKONTROLUJTE, že buňky na sebe navazují a že lze NB postupně projít a jednotlivé buňky nezahlásí chybu.

* Deadline pro odevzdání je stanoven na sředu 21.6.2023.

* Není možno odevzdání opakovat. Odevzdaný úkol zkontroluji a buď zápočet získáte, nebo ne. Je vyžadováno mít alespoň 75% úkolů správně.

* V případě otázek se ptejte veřejně v general kanále v MS Teams. Ostatní mohou mít podobné problémy a tudíž pokud je někde například nejasné zadání, tak ať mají všichni stejnou příležitost.

* Možno psát a odevzdat v Pythonu.